In [1]:
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt
import random as rn


from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,roc_curve,auc

from PIL import Image
from PIL import Image as pil_image
from PIL import ImageDraw

from time import time
from glob import glob
from tqdm import tqdm
from skimage.io import imread
from IPython.display import SVG

from scipy import misc,ndimage
from scipy.ndimage.interpolation import zoom
from matplotlib.pyplot import imread

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Sequential,Input,Model
from tensorflow.keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

In [7]:
# install library for spliting data
!pip install split_folders
import split_folders

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
img_dir = './classification_data/'
split_folder = './classification_split_data/'

In [3]:
if os.path.exists(split_folder)==False:
    os.mkdir(split_folder)
else:
    print('{} already exists'.format(split_folder))

./classification_split_data/ already exists


In [12]:
# split the data in ratio train 80%, test 10%, validation 10%
split_data = True
if split_data == True:
    split_folders.ratio(img_dir, output= split_folder, seed=1337, ratio=(.8, .1, .1))

Copying files: 869 files [00:00, 4360.66 files/s]


In [3]:
x_train_dir = os.path.join(split_folder, 'train')
x_test_dir = os.path.join(split_folder, 'test')
x_val_dir = os.path.join(split_folder, 'val')

print(x_train_dir, x_test_dir, x_val_dir)

./classification_split_data/train ./classification_split_data/test ./classification_split_data/val


In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [6]:
imgsize = 224
batch_size = 32

train_datagen = ImageDataGenerator(rotation_range=10,rescale=1/255,fill_mode="nearest")
valid_datagen = ImageDataGenerator(rotation_range=10,rescale=1/255,fill_mode="nearest") 
test_datagen = ImageDataGenerator(rotation_range=10,rescale=1/255,fill_mode="nearest") 

train_generator = train_datagen.flow_from_directory(
    directory=x_train_dir,
    target_size=(imgsize, imgsize),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
valid_generator = valid_datagen.flow_from_directory(
    directory=x_val_dir,
    target_size=(imgsize,imgsize),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=x_test_dir,
    target_size=(imgsize,imgsize),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 694 images belonging to 3 classes.
Found 85 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


In [7]:
base_model = MobileNetV2(include_top=False,
                  input_shape = (imgsize,imgsize,3),
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = True
    
for layer in base_model.layers:
    print(layer,layer.trainable)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))
model.add(Dense(128))
# model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.summary()


<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f6d75f54100> True
<tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7f6ccbe49f70> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6cc03d44f0> True
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f6cc03d4eb0> True
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f6cc0158f70> True
<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7f6cc0149190> True
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f6cc03a2430> True
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f6cc00bb850> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6cc00a9e20> True
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f6cc00d2df0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f6cc00e1190> True
<te

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 2,422,339
Trainable params: 2,388,227
Non-trainable params: 34,112
_________________________________________________________________


In [8]:
checkpoint = ModelCheckpoint(
    'mobileNetV2Model_224',
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    mode='max',
    save_weights_only=False
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=3,
    verbose=1,
    mode='auto'
)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint,earlystop, reduce]

In [9]:

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [10]:
opt = SGD(lr=1e-4, momentum = 0.99)
opt1 = Adam(lr=1e-3)

model.compile(
    loss=[focal_loss(alpha=.25, gamma=2)],
    optimizer=opt,
    metrics=['accuracy']
)

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=15,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size,
    verbose = 1,
    callbacks=callbacks
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
21/21 [==============================] - ETA: 0s - loss: 0.1852 - accuracy: 0.3308
Epoch 00001: val_accuracy improved from -inf to 0.28125, saving model to mobileNetV2Model_224
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mobileNetV2Model_224/assets
21/21 [==============================] - 40s 2s/step - loss: 0.1852 - accuracy: 0.3308 - val_loss: 0.2640 - val_accuracy: 0.2812 - lr: 1.0000e-04
Epoch 2/15
21/21 [==============================] - ETA: 0s - loss: 0.1557 - accuracy: 0.3943
Epoch 00002: val_accuracy improved from 0.28125 to 0.35938, saving model to mobileNetV2Model_224
INFO:tensorflow:Assets written to: mobileNetV2Model_224/assets
21/21 [==============================] - 32s 2s/step - loss: 0.1557 - accuracy: 0.3943 - val_loss: 0.1539 - val_accuracy: 0.3594 - lr: 1.0000e-04
Epoch 3/15
21/21 [============================

In [14]:
tf.keras.models.save_model(model, 'mobileNetV2Model_224', overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None)

INFO:tensorflow:Assets written to: mobileNetV2Model_224/assets


In [15]:
Y_pred = model.predict_generator(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix
[[ 7  3 12]
 [ 3  5 14]
 [ 0  1 45]]
Classification Report
              precision    recall  f1-score   support

      adding       0.70      0.32      0.44        22
    stirring       0.56      0.23      0.32        22
     unknown       0.63      0.98      0.77        46

    accuracy                           0.63        90
   macro avg       0.63      0.51      0.51        90
weighted avg       0.63      0.63      0.58        90



In [12]:
# Can be skipped. 
model_json = model.to_json()
with open("mobileNetV2Model_224/MobileNetV2model_224.json","w") as json_file:
    json_file.write(model_json)
    
model.save("mobileNetV2Model_224/MobileNetV2model_224.h5")
print("Weights Saved")

Weights Saved


In [13]:
import json
label_map = list(train_generator.class_indices.keys())
with open('labels.txt', 'w') as outfile:
    json.dump(label_map, outfile)

In [25]:
import cv2
frame = cv2.imread('./test_bkp/26.jpg')
print(model.predict_classes(np.float32(np.reshape(cv2.resize(frame,(imgsize,imgsize)),(1,imgsize,imgsize,3)))/255.0 ))

[2]


In [22]:
print(label_map)

['adding', 'stirring', 'unknown']
